In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Create an ImageDataGenerator with validation split
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of data for validation
)

# Flow training data (80% training)
train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Cattle Project (Yohan)/Cow health management/Disease Detection/CNN',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Set as training data
)

# Flow validation data (20% validation)
validation_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Cattle Project (Yohan)/Cow health management/Disease Detection/CNN',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)


Found 850 images belonging to 4 classes.
Found 212 images belonging to 4 classes.


# **Approch 01 - Custom CNN**

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Adjust output layer for 4 classes
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  # Updated for 4 classes
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [5]:
model.fit(train_generator, epochs=20, validation_data=validation_generator)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 395s 12s/step - accuracy: 0.4017 - loss: 2.7603 - val_accuracy: 0.5377 - val_loss: 1.0347
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.5083 - loss: 1.0052 - val_accuracy: 0.6698 - val_loss: 0.9541
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.5713 - loss: 0.9680 - val_accuracy: 0.6509 - val_loss: 0.8818
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.6283 - loss: 0.9212 - val_accuracy: 0.6887 - val_loss: 0.8364
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.5968 - loss: 0.9310 - val_accuracy: 0.6934 - val_loss: 0.7936
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.6530 - loss: 0.8372 - val_accuracy: 0.6698 - val_loss: 0.8079
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.6280 - loss: 0.8800 - val_accuracy: 0.7170 - val_loss: 0.7529
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.6210 - loss: 0.8750 - val_accuracy: 0.7123 - val_loss: 0.7670
E

In [6]:
model.save('cattle_disease_prediction.h5')


# **Approch 02 - RESNET**

In [7]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Load the ResNet50 model, without the top classification layers
resnet_model = ResNet50(include_top=False, input_shape=(128, 128, 3), weights='imagenet')

# Create a Sequential model and add ResNet50 as the base
model_resnet = Sequential()

# Add ResNet50 as the feature extractor
model_resnet.add(resnet_model)

# Flatten the output from ResNet50 and add custom Dense layers
model_resnet.add(Flatten())
model_resnet.add(Dense(128, activation='relu'))
model_resnet.add(Dense(4, activation='softmax'))  # Adjust for 4 output classes

# Compile the model
model_resnet.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_resnet.fit(train_generator, epochs=20, validation_data=validation_generator)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 332s 10s/step - accuracy: 0.5144 - loss: 5.7426 - val_accuracy: 0.4858 - val_loss: 6.8116
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 277s 10s/step - accuracy: 0.6619 - loss: 0.8114 - val_accuracy: 0.4858 - val_loss: 120.3629
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 274s 10s/step - accuracy: 0.6318 - loss: 0.8331 - val_accuracy: 0.4858 - val_loss: 1.1638
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 281s 10s/step - accuracy: 0.6967 - loss: 0.7011 - val_accuracy: 0.4858 - val_loss: 1.3556
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 278s 10s/step - accuracy: 0.7184 - loss: 0.6763 - val_accuracy: 0.4858 - val_loss: 1.5341
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 282s 10s/step - accuracy: 0.7456 - loss: 0.6325 - val_accuracy: 0.4858 - val_loss: 2.2435
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 322s 10s/step - accuracy: 0.7430 - loss: 0.6119 - val_accuracy: 0.4858 - val_loss: 2.4656
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 283s 10s/step

In [9]:
model_resnet.save('cattle_disease_prediction_resnet2.h5')

In [12]:
import numpy as np
import matplotlib.pyplot as plt

# Load a batch of images from the validation set
validation_batch, validation_labels = next(validation_generator)  # get a single batch of validation data

# Get the predicted labels from the model
predicted_labels = model_resnet.predict(validation_batch)

# Convert predicted probabilities to class labels (get index of max probability)
predicted_labels = np.argmax(predicted_labels, axis=1)

# Convert actual one-hot labels to class labels
actual_labels = np.argmax(validation_labels, axis=1)

# Define the number of images to display (16 images in a 4x4 grid)
num_images = 16
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
axes = axes.ravel()

for i in range(num_images):
    axes[i].imshow(validation_batch[i])
    axes[i].set_title(f"Actual: {actual_labels[i]}, Pred: {predicted_labels[i]}")
    axes[i].axis('off')  # Hide axis

# Display the plot
plt.tight_layout()
plt.show()


Output hidden; open in https://colab.research.google.com to view.